# Check dataset action dim

In [13]:
import numpy as np
from lerobot.common.datasets import LeRobotDataset
ds = LeRobotDataset("lerobot/xarm_lift_medium")

ds.download_episodes()

# Print dataset info
print("Dataset info:")
print(f"Number of episodes: {ds.num_episodes}")
print(f"Features: {ds.features}")

Dataset info:
Number of episodes: 800
Features: {'observation.image': {'dtype': 'video', 'shape': (84, 84, 3), 'names': ['height', 'width', 'channel'], 'video_info': {'video.fps': 15.0, 'video.codec': 'av1', 'video.pix_fmt': 'yuv420p', 'video.is_depth_map': False, 'has_audio': False}}, 'observation.state': {'dtype': 'float32', 'shape': (4,), 'names': {'motors': ['motor_0', 'motor_1', 'motor_2', 'motor_3']}}, 'action': {'dtype': 'float32', 'shape': (4,), 'names': {'motors': ['motor_0', 'motor_1', 'motor_2', 'motor_3']}}, 'episode_index': {'dtype': 'int64', 'shape': (1,), 'names': None}, 'frame_index': {'dtype': 'int64', 'shape': (1,), 'names': None}, 'timestamp': {'dtype': 'float32', 'shape': (1,), 'names': None}, 'next.reward': {'dtype': 'float32', 'shape': (1,), 'names': None}, 'next.done': {'dtype': 'bool', 'shape': (1,), 'names': None}, 'index': {'dtype': 'int64', 'shape': (1,), 'names': None}, 'task_index': {'dtype': 'int64', 'shape': (1,), 'names': None}}


In [14]:
# Try to access a single episode first
try:
    # Get the first item from the dataset
    first_item = ds[0]
    print("\nFirst item keys:", first_item.keys())
    
    # Now try to access episodes
    print("\nTrying to access episodes...")
    all_actions = []
    for i in range(min(5, ds.num_episodes)):  # Try first 5 episodes
        item = ds[i]
        if "action" in item:
            all_actions.append(item["action"])
    
    print(f"Successfully collected {len(all_actions)} actions")
except Exception as e:
    print("Error:", str(e))


First item keys: dict_keys(['observation.image', 'observation.state', 'action', 'episode_index', 'frame_index', 'timestamp', 'next.reward', 'next.done', 'index', 'task_index', 'task'])

Trying to access episodes...
Successfully collected 5 actions


objc[57261]: Class AVFFrameReceiver is implemented in both /Users/OAA/miniforge3/envs/robotics/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x11bd083a8) and /Users/OAA/miniforge3/envs/robotics/lib/libavdevice.61.3.100.dylib (0x16bb4c848). One of the two will be used. Which one is undefined.
objc[57261]: Class AVFAudioReceiver is implemented in both /Users/OAA/miniforge3/envs/robotics/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x11bd083f8) and /Users/OAA/miniforge3/envs/robotics/lib/libavdevice.61.3.100.dylib (0x16bb4c898). One of the two will be used. Which one is undefined.


In [18]:
all_actions = []
for episode in ds:
    # print(episode)
    all_actions.append(episode["action"])
    # for step in episode:
    #     print(step)
    #     all_actions.append(step["action"])

actions = np.array(all_actions)

print("Action Mean:", actions.mean(axis=0))
print("Action Std:", actions.std(axis=0))
print("Action Min:", actions.min(axis=0))
print("Action Max:", actions.max(axis=0))

Action Mean: [ 0.27324945 -0.14783773 -0.15354335 -0.23991735]
Action Std: [0.631202   0.6673078  0.6527433  0.65370834]
Action Min: [-1. -1. -1. -1.]
Action Max: [1. 1. 1. 1.]


# Test Diffusion + VAE

In [2]:
from lerobot.common.policies.diffusion.modeling_diffusion import DiffusionPolicy
from lerobot.common.policies.diffusion.configuration_diffusion import DiffusionConfig
import torch

In [3]:
config = DiffusionConfig()
config.latent_dim = 4
config.vae_ckpt = "checkpoints/vae_ckpt.pth"

policy = DiffusionPolicy(config)

policy.vae.load_state_dict(torch.load(config.vae_ckpt))
policy.vae.eval()

ValueError: You must provide at least one image or the environment state among the inputs.